T5 GENERATION & EVALUATION - OUTOFBOX MODEL

Take in test data, get samples n=10 and n=500 using same seed as other experiments. Generate summaries with the out of box model, first with default hyperparameters, and then with tuned hyperparameters.

In [2]:
# mount to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q rouge_score
!pip install -q evaluate
import numpy as np
import evaluate
import rouge_score
import pandas as pd

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.0 MB/s eta 0:00:00


In [10]:
rouge = evaluate.load('rouge')

In [4]:
!pip install transformers
from transformers import AutoTokenizer  , TFAutoModel
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from pprint import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00


In [5]:
# get regular test data
test_file = "drive/MyDrive/266project/billsum_test.csv"
test_data = pd.read_csv(test_file)

In [6]:
#take a sample of 10 for testing my code, then a sample of 500 for the actual experiment
test_data_10 = test_data.sample(n=10, random_state=0)
test_data_500 = test_data.sample(n=500, random_state=0)
test_data_10 = test_data_10.reset_index(drop=True)
test_data_500 = test_data_500.reset_index(drop=True)

In [7]:
# load out of box model
t5_model_name = 't5-base'
orig_t5_model = TFT5ForConditionalGeneration.from_pretrained(t5_model_name)
tokenizer = AutoTokenizer.from_pretrained(t5_model_name)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:
# function that can generate outputs for any test set (except for baseline which will be generated by applying the sumbasic function)

def generate_candidates(orig, model):
  """Test data is a pandas df, "orig" is test_df.orig.
  Default hyperparameters are used.
  """
  input = tokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input)
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
#first on top 10 rows for orig_t5_model(base)
%%timeit
PROMPT = 'summarize: '
test_data_10['t5_out_of_box_candidate'] = test_data_10.apply(lambda x: generate_candidates(PROMPT + x.orig, orig_t5_model), axis = 1)

In [ ]:
t5_rouge_10 = rouge.compute(predictions = test_data_10['t5_out_of_box_candidate'],
                       references= test_data_10['target'])
t5_rouge_10

In [ ]:
# now for the 500 set
%%timeit
test_data_500['t5_out_of_box_candidate'] = test_data_500.apply(lambda x: generate_candidates(PROMPT + x.orig, orig_t5_model), axis = 1)

In [ ]:
t5_rouge_500 = rouge.compute(predictions = test_data_500['t5_out_of_box_candidate'],
                       references= test_data_500['target'])
t5_rouge_500

In [17]:
# run one example to put in appendix
first_example = test_data_10.head(1)
first_example

,Unnamed: 0,orig,target,title
0,2730,SECTION 1. SHORT TITLE.\n\n This Act may be...,Forest Service Financial Accountability Restor...,Forest Service Financial Accountability Restor...


In [18]:
PROMPT = 'summarize: '
first_example['t5_out_of_box_candidate'] = first_example.apply(lambda x: generate_candidates(PROMPT + x.orig, orig_t5_model), axis = 1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:854: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
<ipython-input-18-34cb55d7c513>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_example['t5_out_of_box_candidate'] = first_example.apply(lambda x: generate_candidates(PROMPT + x.orig, orig_t5_model), axis = 1)


In [19]:
pd.set_option('display.max_colwidth', None)

In [20]:
first_example

Unnamed: 0  \
0        2730   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [23]:
# now use new hyperparams
# i tried a few different ones with small outputs to arrive at these

# note - probably should have increased beam size (I thought it would make it slower, but that's not the case)
# make note of that in paper
def generate_candidates_HPs(orig, model):
  """ Test data is a pandas df, "orig" is test_df.orig.
  Tuned hyperparameters are used.
  """
  input = tokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input,
                          max_new_tokens = 150,
                          min_length = 50,
                          num_beams = 1,
                          no_repeat_ngram_size = 2
                          )
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output


In [ ]:
# Used this code to play around with hyperparams

#PROMPT = 'summarize: '
#test_data_10['t5_out_of_box_candidate_HPs'] = test_data_10.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, orig_t5_model), axis = 1)

In [ ]:
%%timeit
PROMPT = 'summarize: '
test_data_500['t5_out_of_box_candidate_HPs'] = test_data_500.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, orig_t5_model), axis = 1)

In [ ]:
test_data_500.fillna("", inplace=True)

In [ ]:
t5_rouge_500 = rouge.compute(predictions = test_data_500['t5_out_of_box_candidate_HPs'],
                       references= test_data_500['target'])
t5_rouge_500

In [ ]:
#test_data_500.to_csv("drive/MyDrive/266project/500_with_t5_raw_hp_candidates.csv")